In [1]:
from pyspark.sql import SparkSession

# New API
spark_session = SparkSession.builder\
        .master("spark://192.168.2.133:7077") \
        .appName("simpleAnalysisComputationTime")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.dynamicAllocation.shuffleTracking.enabled",True)\
        .config("spark.shuffle.service.enabled", False)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.executor.cores", 4)\
        .config("spark.driver.port",9999)\
        .config("spark.blockManager.port",10005)\
        .getOrCreate()

# Old API (RDD)
spark_context = spark_session.sparkContext

spark_context.setLogLevel("WARN")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/15 18:22:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df = spark_session.read.json('hdfs://192.168.2.133:9000/user/ubuntu/yasp_chunk_10000_output/*')

In [3]:
from pyspark.sql.functions import col, count, when

# Calclulate the times of wins each team
radiant_wins = df.select(col("radiant_win")).where(col("radiant_win")).count()
dire_wins = df.select(col("radiant_win")).where(~col("radiant_win")).count()

# Victory percentage
total_games = df.count()
radiant_win_rate = radiant_wins / total_games
dire_win_rate = dire_wins / total_games


win_rate_diff = abs(radiant_win_rate - dire_win_rate) * 100


print(f"Radiant win rate: {radiant_win_rate:.2%}")
print(f"Dire win rate: {dire_win_rate:.2%}")
print(f"Winning percentage gap: {win_rate_diff:.2f}%")


Radiant win rate: 51.88%
Dire win rate: 48.06%
Winning percentage gap: 3.82%


In [4]:
from pyspark.sql.functions import col, when, size
#confirm
df = df.withColumn("has_10_minutes", size(col("radiant_gold_adv")) >= 10)

# Filter
df = df.filter(col("has_10_minutes"))

# Extract
df = df.withColumn("radiant_gold_adv_10min", col("radiant_gold_adv")[9]) \
       .withColumn("radiant_xp_adv_10min", col("radiant_xp_adv")[9])

radiant_gold_lead = df.filter(col("radiant_gold_adv_10min") > 0)
dire_gold_lead = df.filter(col("radiant_gold_adv_10min") < 0)

radiant_xp_lead = df.filter(col("radiant_xp_adv_10min") > 0)
dire_xp_lead = df.filter(col("radiant_xp_adv_10min") < 0)

# Calculate win rates
radiant_gold_lead_win_rate = radiant_gold_lead.filter(col("radiant_win") == True).count() / radiant_gold_lead.count()
dire_gold_lead_win_rate = dire_gold_lead.filter(col("radiant_win") == False).count() / dire_gold_lead.count()

radiant_xp_lead_win_rate = radiant_xp_lead.filter(col("radiant_win") == True).count() / radiant_xp_lead.count()
dire_xp_lead_win_rate = dire_xp_lead.filter(col("radiant_win") == False).count() / dire_xp_lead.count()

print(f"Radiant 10-minute gold lead win rate: {radiant_gold_lead_win_rate * 100:.2f}%")
print(f"Dire 10-minute gold lead win rate: {dire_gold_lead_win_rate * 100:.2f}%")
print(f"Radiant 10-minute XP lead win rate: {radiant_xp_lead_win_rate * 100:.2f}%")
print(f"Dire 10-minute XP lead win rate: {dire_xp_lead_win_rate * 100:.2f}%")


Radiant 10-minute gold lead win rate: 66.37%
Dire 10-minute gold lead win rate: 63.10%
Radiant 10-minute XP lead win rate: 65.41%
Dire 10-minute XP lead win rate: 61.73%


In [5]:
from pyspark.sql.functions import explode, sum as spark_sum

df_exploded = df.withColumn("players", explode("players"))

df_kills = df_exploded.groupBy("match_id").agg(spark_sum("players.kills").alias("total_kills"))

# Find the match with the highest total kills
highest_kills_match = df_kills.orderBy("total_kills", ascending=False).first()

highest_kills_match_id = highest_kills_match["match_id"]
highest_kills = highest_kills_match["total_kills"]

print(f"Match with the highest total kills: {highest_kills_match_id}, Total kills: {highest_kills}")


Match with the highest total kills: 1998243598, Total kills: 268


In [6]:
from pyspark.sql.functions import col
longest_matches = df.orderBy(col("duration").desc()).select("match_id", "duration").limit(5)

shortest_matches = df.where(col("duration") > 0).orderBy("duration").select("match_id", "duration").limit(5)

print("Five Longest Matches:")
longest_matches.show()

print("Five Shortest Matches:")
shortest_matches.show()

Five Longest Matches:


+----------+--------+
|  match_id|duration|
+----------+--------+
|1998243598|    9318|
|1984881220|    8806|
|1982514589|    7479|
|1988285140|    7242|
|1995768838|    7052|
+----------+--------+

Five Shortest Matches:


+----------+--------+
|  match_id|duration|
+----------+--------+
|1980174434|     304|
|1971139458|     304|
|1965719668|     306|
|1959749569|     308|
|1984245435|     309|
+----------+--------+



In [7]:
spark_context.stop()